In [1]:
## 使用Unstructured File读取text文件、powerpoints、html、PDF、images或其他需要安装以下
## windows安装detectron诸多问题，可参考：
#####  - https://blog.csdn.net/u010674979/article/details/125719919
#####  - http://www.taodudu.cc/news/show-1215672.html

# # Install package
# !pip install "unstructured[local-inference]"
# !pip install "detectron2@git+https://github.com/facebookresearch/detectron2.git@v0.6#egg=detectron2"
# !pip install layoutparser[layoutmodels,tesseract]

In [2]:
# # Install other dependencies
# # https://github.com/Unstructured-IO/unstructured/blob/main/docs/source/installing.rst
# !brew install libmagic
# !brew install poppler
# !brew install tesseract
# # If parsing xml / html documents:
# !brew install libxml2
# !brew install libxslt

'brew' is not recognized as an internal or external command,
operable program or batch file.
'brew' is not recognized as an internal or external command,
operable program or batch file.
'brew' is not recognized as an internal or external command,
operable program or batch file.
'brew' is not recognized as an internal or external command,
operable program or batch file.
'brew' is not recognized as an internal or external command,
operable program or batch file.


In [1]:
!pwd

/f/AI/hohoChat


In [2]:
cd langchain-ChatGLM-master/

F:\AI\hohoChat\langchain-ChatGLM-master


In [3]:
# !pip install -r requirements.txt -i https://mirror.sjtu.edu.cn/pypi/web/simple

In [4]:
!cp -r nltk_data ../../

In [5]:
from IPython.display import display, Markdown, clear_output

def display_answer(agent, query, vs_path, history = []):
    for resp, history in local_doc.qa.get_knowledge_based_answer(query = query,
                                                                  vs_path = vs_path,
                                                                  chat_history = history,
                                                                  streaming = True):
        clear_output(wait = True)
        display(Markdown(resp['result']))
    return resp, history

In [6]:
import torch.cuda
import torch.backends

from configs import model_config

model_config.embedding_model_dict = {
    "ernie-tiny": "nghuyong/ernie-3.0-nano-zh",
    "ernie-base": "nghuyong/ernie-3.0-base-zh",
    "text2vec-base": "shibing624/text2vec-base-chinese",
    "text2vec": "/home/mw/input/text2vec2538",   # hoho: 注意路径
}

model_config.llm_model_dict = {
    "chatyuan": "ClueAI/ChatYuan-large-v2",
    "chatglm-6b-int4-qe": "THUDM/chatglm-6b-int4-qe",
    "chatglm-6b-int4": "THUDM/chatglm-6b-int4",
    "chatglm-6b-int8": "THUDM/chatglm-6b-int8",
    "chatglm-6b": "/home/mw/input/ChatGLM6B6449",  # hoho: 注意路径
}

model_config.VS_ROOT_PATH = "../../temp"



In [7]:
from chains.local_doc_qa import LocalDocQA

EMBEDDING_MODEL = "text2vec-base"
VECTOR_SEARCH_TOP_K = 6
LLM_MODEL = "chatglm-6b-int4"
LLM_HISTORY_LEN = 3
DEVICE = "cuda" if torch.cuda.is_available() else "mps" if torch.backends.mps.is_available() else "cpu"

local_doc_qa = LocalDocQA()

local_doc_qa.init_cfg(llm_model = LLM_MODEL,
                     embedding_model = EMBEDDING_MODEL,
                     llm_history_len = LLM_HISTORY_LEN,
                     top_k = VECTOR_SEARCH_TOP_K)

Explicitly passing a `revision` is encouraged when loading a model with custom code to ensure no malicious code has been contributed in a newer revision.
Explicitly passing a `revision` is encouraged when loading a configuration with custom code to ensure no malicious code has been contributed in a newer revision.
Explicitly passing a `revision` is encouraged when loading a model with custom code to ensure no malicious code has been contributed in a newer revision.


No compiled kernel found.
Compiling kernels : C:\Users\Administrator\.cache\huggingface\modules\transformers_modules\THUDM\chatglm-6b-int4\f6b88da8c13be209fdaa3bfe2d3099563947a0ca\quantization_kernels_parallel.c
Compiling gcc -O3 -fPIC -pthread -fopenmp -std=c99 C:\Users\Administrator\.cache\huggingface\modules\transformers_modules\THUDM\chatglm-6b-int4\f6b88da8c13be209fdaa3bfe2d3099563947a0ca\quantization_kernels_parallel.c -shared -o C:\Users\Administrator\.cache\huggingface\modules\transformers_modules\THUDM\chatglm-6b-int4\f6b88da8c13be209fdaa3bfe2d3099563947a0ca\quantization_kernels_parallel.so
Compile default cpu kernel failed, using default cpu kernel code.
Compiling gcc -O3 -fPIC -std=c99 C:\Users\Administrator\.cache\huggingface\modules\transformers_modules\THUDM\chatglm-6b-int4\f6b88da8c13be209fdaa3bfe2d3099563947a0ca\quantization_kernels.c -shared -o C:\Users\Administrator\.cache\huggingface\modules\transformers_modules\THUDM\chatglm-6b-int4\f6b88da8c13be209fdaa3bfe2d3099563

In [ ]:
vs_path, _ = local_doc_qa.init_knowledge_vector_store("F:/AI/hohoChat/data/d2l-zh-pytorch.pdf")
vs_path

In [10]:
!pwd

/f/AI/hohoChat/langchain-ChatGLM-master
